In [28]:
!pip install llama_index
!pip install llama-index-embeddings-gemini
!pip install llama-index-llms-gemini
!pip install google-generativeai

In [29]:
from google.colab import auth
import os
from google.colab import userdata

auth.authenticate_user()
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [30]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core import Document
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import TokenTextSplitter
import requests


In [31]:
tsPracticesDoc = "https://raw.githubusercontent.com/AhsanAyaz/gemini-rag-llamaindex-ts/refs/heads/main/data-sources/typescript_best_practices.txt"
response = requests.get(tsPracticesDoc)
if response.status_code == 200:
    content = response.text
else:
    raise Exception(f"Failed to download the file from {url}")

documents = [Document(text=content)]

In [32]:
llm = Gemini(model_name="models/gemini-1.5-flash-latest")
embed_model = GeminiEmbedding(model_name="models/embedding-001", generation_config = {"temperature": 0.7, "topP": 0.8, "topK": 40})

text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=20)

# global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [33]:
index = VectorStoreIndex.from_documents(documents, show_progress=True)

query_engine = index.as_query_engine()

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
response = query_engine.query("What are the benefits of DRY in TypeScript? Can you give some examples and give some code samples?")
print(response)

DRY (Don't Repeat Yourself) is a principle that encourages developers to avoid repeating code. This leads to more maintainable and efficient code. 

For example, instead of writing the same logic multiple times, you can create a function or a reusable component that encapsulates that logic. This way, if you need to make changes, you only need to modify the function or component once, and the changes will be reflected everywhere it's used.

Here's an example:

```typescript
// Bad Example: Repeating logic
let area1 = 10 * 20;
let area2 = 15 * 30; 

// Good Example: DRY principle applied
function calculateArea(width: number, height: number): number {
  return width * height;
}

let area1 = calculateArea(10, 20);
let area2 = calculateArea(15, 30);
```

In this example, the `calculateArea` function encapsulates the logic for calculating the area of a rectangle. This makes the code more readable and maintainable. If you need to change the way the area is calculated, you only need to modify 